# predict_future_output 

## Goal: predict future outputs of the various electron gun beams


In [31]:
input_file = "dataset1_clean.csv"

In [32]:
# imports:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
# import utils and other models
import utils
import bla_avg_model
import bla_constant_model
import bla_linear_model
import bla_lasso_model

In [33]:
# load in cleaned dataset
data = pd.read_csv(input_file, index_col=[0], skipinitialspace=True)

# add in integer index
data = data.reset_index().reset_index().set_index(["Time"])
print(data.head())

                     index  DAC_S0_95th_quantile  DAC_S0_median  \
Time                                                              
1900-01-01 00:00:00      0                   2.0            2.0   
1900-01-01 00:01:00      1                   2.0            2.0   
1900-01-01 00:02:00      2                   2.0            2.0   
1900-01-01 00:03:00      3                   2.0            2.0   
1900-01-01 00:04:00      4                   2.0            2.0   

                     DAC_S0_5th_quantile  SO_(volts)_95th_quantile  \
Time                                                                 
1900-01-01 00:00:00                  2.0                       2.0   
1900-01-01 00:01:00                  2.0                       2.0   
1900-01-01 00:02:00                  2.0                       2.0   
1900-01-01 00:03:00                  2.0                       2.0   
1900-01-01 00:04:00                  2.0                       2.0   

                     SO_(volts)_median 

In [34]:
# (year set to 1900 since that is the base for datetime objects)
# note date ranges are not inclusive
train_date_range = ["1900-01-01", "1900-01-08"]
test_date_range = ["1900-01-08", "1900-01-10"]

full_train, test = utils.split_by_day(data, train_date_range, test_date_range)

#print(full_train.info())
#print(test.info())

In [35]:
# split full_train into a train and valdiation set
validation_frac = .1
train, validation = train_test_split(full_train, test_size=validation_frac)
#print(validation.info())
#print(train.info())

In [36]:
# convert the train and validation dataframes into a set of np arrays that can be more readily used to train models
train_X_95th, train_X_median, train_X_5th = utils.split_X_into_quantiles(train, accept_index=True)
train_Y_95th, train_Y_median, train_Y_5th = utils.split_Y_into_beams_and_quantiles(train, )

validation_X_95th, validation_X_median, validation_X_5th = utils.split_X_into_quantiles(validation, accept_index=True)
validation_Y_95th, validation_Y_median, validation_Y_5th = utils.split_Y_into_beams_and_quantiles(validation)

### Evaluation Metrics

Mean Squared Error (MSE) computed using sklearn.metrics.mean_squared_error

In [37]:
def prop_within_bounds(actual, pred_95th, pred_5th):
    # return the proportion of points in actual that fall between the predicted 5th and 95th quantiles
    num_points = actual.shape[0]
    within_upper = actual > pred_95th
    within_lower = actual < pred_5th
    within_range = np.logical_and(within_upper, within_lower)
    return np.count_nonzero(within_range) / num_points

## Baseline 1: BlaConstantModel

This model averages the input for each beam over the training period and predicts that value as constant for each beam.

This represents the most naive model, since it assumes indpendence from the features and labels, and no change in the 
labels over time.

### Train Baseline 1

In [38]:
baseline_1_95 = bla_constant_model.BlaConstantModel()
baseline_1_median = bla_constant_model.BlaConstantModel()
baseline_1_5 = bla_constant_model.BlaConstantModel()

baseline_1_95.train(train_X_median, train_Y_95th)
baseline_1_median.train(train_X_median, train_Y_median)
baseline_1_5.train(train_X_median, train_Y_5th)

baseline_1_pred_95 = [baseline_1_95.predict(validation_X_median, i) for i in range(6)]
baseline_1_pred_median = [baseline_1_median.predict(validation_X_median, i) for i in range(6)]
baseline_1_pred_5 = [baseline_1_5.predict(validation_X_median, i) for i in range(6)]


### Validate Baseline 1

In [39]:
baseline_1_valid_props = []
for i in range(6):
    p = prop_within_bounds(validation_Y_median[i], baseline_1_pred_95[i], baseline_1_pred_5[i])
    baseline_1_valid_props.append(p)
    
print(baseline_1_valid_props)

baseline_1_valid_mses = []
for i in range(6):
    mse_95 = mean_squared_error(validation_Y_95th[i], baseline_1_pred_95[i])
    mse_median = mean_squared_error(validation_Y_median[i], baseline_1_pred_median[i])
    mse_5 = mean_squared_error(validation_Y_5th[i], baseline_1_pred_5[i])
    baseline_1_valid_mses.append([mse_95, mse_median, mse_5])
    
print(baseline_1_valid_mses)

[0.29563492063492064, 0.5, 0.18154761904761904, 0.4662698412698413, 0.18353174603174602, 0.3541666666666667]
[[0.04297657680959674, 0.04249573580770108, 0.04229091080001955], [0.03228432128906246, 0.03220416945004089, 0.032305414847883476], [0.24817932006401927, 0.24346601643695664, 0.2380120790028731], [0.02800162907117755, 0.028038107969078376, 0.02852089620929389], [0.11915179795203489, 0.11790977423514339, 0.1162519840610098], [0.06486270715972631, 0.06132761951851131, 0.05926721699264974]]


## Baseline 2: BlaAvgModel
This model ignores differences in beams and tries to fit a single model for all beams by averaging the output from all
beams into a single "y" before fitting.

### Train Baseline 2

In [40]:
baseline_2_95 = bla_avg_model.BlaAvgModel()
baseline_2_median = bla_avg_model.BlaAvgModel()
baseline_2_5 = bla_avg_model.BlaAvgModel()

baseline_2_95.train(train_X_median, train_Y_95th)
baseline_2_median.train(train_X_median, train_Y_median)
baseline_2_5.train(train_X_median, train_Y_5th)

baseline_2_pred_95 = baseline_2_95.predict(validation_X_median)
baseline_2_pred_median = baseline_2_median.predict(validation_X_median)
baseline_2_pred_5 = baseline_2_5.predict(validation_X_median)

### Validate Baseline 2

In [41]:
baseline_2_valid_props = []
for i in range(6):
    p = prop_within_bounds(validation_Y_median[i], baseline_2_pred_95, baseline_2_pred_5)
    baseline_2_valid_props.append(p)
    
print(baseline_2_valid_props)

baseline_2_valid_mses = []
for i in range(6):
    mse_95 = mean_squared_error(validation_Y_95th[i], baseline_2_pred_95)
    mse_median = mean_squared_error(validation_Y_median[i], baseline_2_pred_median)
    mse_5 = mean_squared_error(validation_Y_5th[i], baseline_2_pred_5)
    baseline_2_valid_mses.append([mse_95, mse_median, mse_5])
    
print(baseline_2_valid_mses)

[0.0, 0.002976190476190476, 0.0, 0.0, 0.49107142857142855, 0.0]
[[17.908584978970893, 17.887450097053435, 17.87296108730482], [0.3708908003354619, 0.3313506433570611, 0.294005333267374], [0.8478163592743483, 0.8235454535140354, 0.8041406438456409], [13.120875592853777, 13.35242021889199, 13.572075628805047], [0.05611916857392629, 0.06046656102494984, 0.0647611110364742], [0.8820692658925832, 0.9008535418823075, 0.9214694182091105]]


## Model 1: Regular Least Squares Regression for Each Beam
This model trains a regular least squares model for each beam independtly. Parameters for the least squares model are
standard, and an intercept is added.

### Train Model 1

In [42]:
model_1_95 = bla_linear_model.BlaLinearModel()
model_1_median = bla_linear_model.BlaLinearModel()
model_1_5 = bla_linear_model.BlaLinearModel()

model_1_95.train(train_X_median, train_Y_95th)
model_1_median.train(train_X_median, train_Y_median)
model_1_5.train(train_X_median, train_Y_5th)

model_1_pred_95 = [model_1_95.predict(validation_X_median, i) for i in range(6)]
model_1_pred_median = [model_1_median.predict(validation_X_median, i) for i in range(6)]
model_1_pred_5 = [model_1_5.predict(validation_X_median, i) for i in range(6)]

### Validate Model 1


In [53]:
model_1_valid_props = []
for i in range(6):
    p = prop_within_bounds(validation_Y_median[i], model_1_pred_95[i], model_1_pred_5[i])
    model_1_valid_props.append(p)
    print(model_1_median.models[i].score(validation_X_median, validation_Y_median[i]))
    
print(model_1_valid_props)

model_1_valid_mses = []
for i in range(6):
    mse_95 = mean_squared_error(validation_Y_95th[i], model_1_pred_95[i])
    mse_median = mean_squared_error(validation_Y_median[i], model_1_pred_median[i])
    mse_5 = mean_squared_error(validation_Y_5th[i], model_1_pred_5[i])
    model_1_valid_mses.append([mse_95, mse_median, mse_5])
    
print(model_1_valid_mses)

0.8204417073703527
0.8355606678091911
0.9500814103118561
0.8891197457422311
0.9751441389701442
0.6898552459074605
[0.7767857142857143, 0.9315476190476191, 0.7321428571428571, 0.7916666666666666, 0.8125, 0.5942460317460317]
[[0.007968462831483381, 0.007623441583320466, 0.007802038600033137], [0.0055223350816327186, 0.005293504793491854, 0.005954430589633759], [0.013987724142431558, 0.012152500723241217, 0.01177437061216488], [0.0031856661707764405, 0.003108799560535043, 0.0032621611619321005], [0.003166795002830785, 0.00293056313871252, 0.0028289609790437], [0.02057199943655961, 0.019017032058222087, 0.01836681503629312]]


## Model 2: Lasso (Linear Regression with L1 Regularization)
This model adds a l1 regularization term to the objective function. This has the effect of penalizing nonzero coefficients,
which in turn minimizes the number of nonzero coefficients. 

### Train Model 2

In [55]:
model_2_95 = bla_lasso_model.BlaLassoModel(alpha=1)
model_2_median = bla_lasso_model.BlaLassoModel(alpha=1)
model_2_5 = bla_lasso_model.BlaLassoModel(alpha=1)

model_2_95.train(train_X_median, train_Y_95th)
model_2_median.train(train_X_median, train_Y_median)
model_2_5.train(train_X_median, train_Y_5th)

model_2_pred_95 = [model_2_95.predict(validation_X_median, i) for i in range(6)]
model_2_pred_median = [model_2_median.predict(validation_X_median, i) for i in range(6)]
model_2_pred_5 = [model_2_5.predict(validation_X_median, i) for i in range(6)]

### Validate Model 2

In [56]:
model_2_valid_props = []
for i in range(6):
    p = prop_within_bounds(validation_Y_median[i], model_2_pred_95[i], model_2_pred_5[i])
    model_2_valid_props.append(p)
    print(model_2_median.models[i].coef_, model_2_median.models[i].intercept_)
    print(model_2_median.models[i].score(validation_X_median, validation_Y_median[i]))
    
print(model_2_valid_props)

model_2_valid_mses = []
for i in range(6):
    mse_95 = mean_squared_error(validation_Y_95th[i], model_2_pred_95[i])
    mse_median = mean_squared_error(validation_Y_median[i], model_2_pred_median[i])
    mse_5 = mean_squared_error(validation_Y_5th[i], model_2_pred_5[i])
    model_2_valid_mses.append([mse_95, mse_median, mse_5])
    
print(model_2_valid_mses)

[ 0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -5.81353376e-02
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  3.87670173e-05] [-40.75561136]
0.5888872562787588
[ 0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00

In [46]:
scatter_indices = list(range(len(baseline_pred_95)))
#plt.plot(pred_95, color="Blue")
plt.scatter(scatter_indices, baseline_pred_95, label="95th")
plt.scatter(scatter_indices, baseline_pred_5, c="Red", label="5th")
plt.scatter(scatter_indices, validation_Y_median[0], c="Yellow", label="median")
plt.scatter(scatter_indices, validation_Y_median[4], c="Green", label="median")
plt.scatter(scatter_indices, validation_Y_median[1], c="Purple", label="median")
plt.scatter(scatter_indices, validation_Y_median[2], c="Orange", label="median")
plt.show()

NameError: name 'baseline_pred_95' is not defined

In [ ]:
plt.plot(baseline_pred_5, color="Red")
plt.show()



In [ ]:
# evaluate models:
# split test set into a set of np arrays that can be more readily used to generate model predictions
test_X_95th, test_X_median, test_X_5th = utils.split_X_into_quantiles(test)
test_Y_95th, test_Y_median, test_Y_5th = utils.split_Y_into_beams_and_quantiles(test)